# Claim Extraction Experiment

In [21]:
import os
from google import genai
from google.genai import types
from src.utils.loader_googlefile import load_video_file_from_metadata, save_video_file_metadata

from src.modules.NewsMatcher import NewsMatcher

import time


In [22]:
example = {
    'claim': 'Comerciantes e empresários expressam gratidão ao Presidente Lula por um novo decreto que reduz a intermediação na cadeia alimentar, prometendo melhores preços para os consumidores e impulsionando diversos setores do comércio no Brasil.', 'context': 'Aqui está a extração do vídeo:\n\n*Principais tópicos discutidos:\n\n1.  **Redução da Intermediação:* O orador principal agradece ao presidente por diminuir a intermediação na cadeia produtiva, o que impacta diretamente a alimentação do povo.\n2.  *Melhores Preços e Competitividade:* A medida proporciona melhores condições de competição para os produtores e permite o repasse de melhores preços para supermercados e consumidores.\n3.  *Atendimento à Demanda Alimentar:* O objetivo final e o resultado da medida é atender à demanda de alimentação da população brasileira e dos trabalhadores.\n4.  *Impacto Positivo em Vários Setores:* A mudança é descrita como histórica e benéfica para mercadistas (grandes, pequenos e médios), pequenos restaurantes, padarias e vendedores de hortifrúti em todo o Brasil.\n5.  *Benefício Geral para o Brasil:* O presidente Lula reforça que a medida é boa para todos esses setores e, consequentemente, para o trabalhador e para o país como um todo.\n\n*Participantes do vídeo e tom do discurso:\n\n   *Orador Principal (0:00 - 1:07):* Um homem de terno, com um microfone, dirigindo-se ao presidente.\n    *   *Tom:* Extremamente agradecido, elogioso e entusiasmado. Ele usa palavras como "agradecer muito", "trabalho incrível", "mudança histórica" e "passo extraordinário", enfatizando a importância e o impacto positivo da medida para diversos setores do comércio e para o consumidor brasileiro.\n*   *Presidente Luiz Inácio Lula da Silva (1:07 - 1:39):* O presidente do Brasil, também com um microfone, respondendo ao orador.\n    *   *Tom:* Confirmatório, inclusivo e otimista. Ele ecoa a aprovação da medida, listando os beneficiários (supermercados, restaurantes, padarias, hortifrúti de todos os portes) e concluindo que, se é bom para todos, é bom para o trabalhador e para o Brasil. A sua linguagem é mais direta e abrangente.\n*   *Outros Participantes (1:40):* No final, vê-se o presidente Lula e o vice-presidente Geraldo Alckmin, junto com os outros oradores, segurando documentos, o que indica um evento formal de assinatura ou sanção de um decreto.\n\nEm resumo, o vídeo registra um momento de grande celebração e gratidão por parte dos representantes do comércio e do setor alimentar, com o presidente confirmando o alcance e os benefícios da medida para a população e a economia brasileira.', 'message': ''
}

In [23]:
client = genai.Client()

In [24]:
from dataclasses import dataclass

@dataclass
class Subclaim:
    claim_text: str
    evidence_types: list[str]
    query: list[str]
    verification_result: str | None = None
    justification: str | None = None

In [25]:
n = 4
claim = example['claim']

prompt = f"""
Você é um verificador de fatos.
Afirmação: "{claim}"

Gere {n} sub-afirmações testáveis que, se verificadas, provariam ou refutariam a afirmação original.
Para cada sub-afirmação:
- Especifique que tipo de evidência a confirmaria ou refutaria (dados, testemunho, documentação, registro temporal)
- Forneça a query de busca que recuperaria essa evidência.

Somente retorne as perguntas em formato de lista com suas respectivas justificativas, sem qualquer texto adicional, com o seguinte formato:
Sub-afirmação: [asserção específica]
Tipo de evidência: [o que provaria/refutaria]
Query de busca: [consulta para recuperar a evidência, ex: "<query 1>", "<query 2>", ...]

Priorize sub-afirmações por:
1. Observabilidade direta
2. Menor número de passos inferenciais
3. Disponibilidade de fontes primárias"""

# Generate the summary
print("Generating summary...")
response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[
        prompt
    ],
    config=types.GenerateContentConfig(
        temperature=0.2
    )
)

sub_claims: list[Subclaim] = []

for item in response.text.split('\n\n'):
    lines = item.strip().split('\n')
    if len(lines) < 3:
        continue

    subclaim_text = lines[0].replace('Sub-afirmação: ', '').strip()
    evidence_type = lines[1].replace('Tipo de evidência: ', '').strip()
    query_text = lines[2].replace('Query de busca: ', '').strip()

    sub_claims.append(Subclaim(
        claim_text=subclaim_text,
        evidence_types=evidence_type.split(', '),
        query=[q.strip().strip('".') for q in query_text.split(',')]
    ))


12/19/2025 03:12:27 AM - AFC is enabled with max remote calls: 10.


Generating summary...


12/19/2025 03:12:40 AM - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


In [26]:
sub_claims

[Subclaim(claim_text='Existe um novo decreto presidencial assinado por Lula que visa especificamente reduzir a intermediação na cadeia alimentar no Brasil.', evidence_types=['Documentação', 'Registro temporal'], query=['decreto presidencial Lula cadeia alimentar intermediação', 'Diário Oficial da União decreto Lula alimentos', 'legislação Lula redução intermediários alimentos', 'novo decreto Lula agricultura'], verification_result=None, justification=None),
 Subclaim(claim_text='Entidades representativas de comerciantes e empresários (ex: confederações, federações, associações setoriais) emitiram comunicados ou declarações públicas de gratidão ou apoio ao Presidente Lula especificamente por um decreto que reduz a intermediação na cadeia alimentar.', evidence_types=['Testemunho', 'Documentação'], query=['CNC agradece decreto Lula cadeia alimentar', 'Fecomércio apoio decreto Lula alimentos', 'CNA posicionamento decreto intermediação Lula', 'Associações empresariais gratidão Lula decreto 

In [27]:
news_matcher = NewsMatcher()

In [28]:
news_matcher = NewsMatcher()

queries: list[str] = []
for sub_claim in sub_claims:
    queries.extend(sub_claim.query)

news = news_matcher.run(queries)

In [29]:
news

[{'source': 'GNews',
  'score': 0.3285194751675468,
  'title': 'O novo decreto de Lula que bolsonaristas tentam derrubar no Congresso - CartaCapital',
  'url': 'https://news.google.com/rss/articles/CBMisAFBVV95cUxPanFhREYwSFdfQWE0dEc5NEUwTk44NXhOQV9QR3VLMGNvVmtWN0MtWTUwUU1WeVh2NE5tdHhSUVAxbUNCNHRBeHhXd1JOaFl4M1pOaHo3Q3RhRlRBSDVYWVhPNmI5ekpnRDl6Tzd4ZGZ0VGFza0lfUzNvdzEwQmFaWERQMTByaFVYRkdzUXAxYzNsWG81NFgtbExVbzRzd3JhOHJQaExaS3pGSmF0NGZTUg?oc=5&hl=en-US&gl=US&ceid=US:en',
  'query': 'novo decreto Lula agricultura',
  'description': 'O novo decreto de Lula que bolsonaristas tentam derrubar no Congresso  CartaCapital'},
 {'source': 'GNews',
  'score': 0.271792477019343,
  'title': 'MST pode ser beneficiado com novo decreto de Lula, aponta bancada ruralista - Gazeta do Povo',
  'url': 'https://news.google.com/rss/articles/CBMipgFBVV95cUxOMnhqb25zM0lIYVAyOGZpb09DNi1jZFN3eWFEQUFyVnRzUVdyLUk2NGxFdHd1WTJybGtiZ29MYjRSQ2lWZm5obzdWWFduQ094RWlwLXRpdFh5QjZlNEItVGpWby1BU0V2cTQyOTFxZHB6X2xNZks2b2drMzNU

In [30]:
afirmations = [
    f"Afirmação {i}: \"{subclaim.claim_text}\""
    for i, subclaim in enumerate(sub_claims, 1)
]

news_context = [
    f"Notícia {i}: \"{article['title']}\""
    for i, article in enumerate(news, 1)
]

context = example['context']

prompt = f"""
Você é um verificador de fatos e analista de contexto.
Dado o seguinte contexto, notícias e afirmações, determine se a afirmação é suportada, refutada ou não pode ser verificada.
Caso o a afirmação seja suportada

Contexto: "{context}"
{'\n'.join(news_context)}
{'\n'.join(afirmations)}

Responda para cada uma das afirmações apenas com uma das seguintes frases:
- Suportada Além do Contexto
- Potenciamente Suportada Além do Contexto
- Suportada Apenas pelo Contexto
- Suportada Fracamente pelo Contexto
- Refutada pelo contexto
- Refutada Além do Contexto
- Não verificável
no seguinte formato:
Afirmação 1: [resultado]
Justificativa: [breve explicação]
Afirmação 2: [resultado]
Justificativa: [breve explicação]
...
"""

response = client.models.generate_content(
    model='gemini-2.5-flash',
    contents=[prompt],
    config=types.GenerateContentConfig(
        temperature=0.2
    )
)



12/19/2025 03:13:30 AM - AFC is enabled with max remote calls: 10.
12/19/2025 03:13:41 AM - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


In [37]:
sub_claims_text = [
    f"Afirmação {i+1}: \"{subclaim.claim_text}\"\n"
        f"Veredito: {subclaim.verification_result or 'N/A'}\n"
    f"Justificativa: {subclaim.justification or 'N/A'}\n"
    for i, subclaim in enumerate(sub_claims)
]

prompt = f"""
Você é um especialista em comunicação de desinformação.
Dado o seguinte conjunto de sub-afirmações verificadas e notícias relacionadas, resuma as justificativas em mensagens curtas e objetivas para envio para o usuário.
Mantenha um tom neutro e cético e forneça apenas fatos e evidências relevantes, evitando julgamentos ou opiniões pessoais.

{'\n'.join(sub_claims_text)}

Após analisar as justificativas, gere um resumo compacto sobre a veracidade da afirmação original, destacando os pontos principais de suporte ou refutação:
{claim}

Formate em texto simples, cada mensagem em uma linha separada, com no máximo 500 caracteres cada.
"""

In [ ]:
response = client.models.generate_content(
    model=self.model,
    contents=[prompt],
    config=types.GenerateContentConfig(
        temperature=0.2
    )
)


Você é um especialista em comunicação de desinformação.
Dado o seguinte conjunto de sub-afirmações verificadas e notícias relacionadas, resuma as justificativas em mensagens curtas e objetivas para envio para o usuário.
Mantenha um tom neutro e cético e forneça apenas fatos e evidências relevantes, evitando julgamentos ou opiniões pessoais.

Afirmação 1: "Existe um novo decreto presidencial assinado por Lula que visa especificamente reduzir a intermediação na cadeia alimentar no Brasil."
Veredito: Suportada Além do Contexto
Justificativa: O contexto do vídeo indica fortemente a assinatura de um decreto com o objetivo de reduzir a intermediação na cadeia alimentar. As Notícias 1 e 2 confirmam a existência de um "novo decreto de Lula", corroborando a afirmação.

Afirmação 2: "Entidades representativas de comerciantes e empresários (ex: confederações, federações, associações setoriais) emitiram comunicados ou declarações públicas de gratidão ou apoio ao Presidente Lula especificamente po